In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

In [2]:
from utils import load_monk_data, preprocess_monk
df = load_monk_data(1)
X, Y = preprocess_monk(df)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

In [3]:
import torch
from collections import OrderedDict

class NN_BinClassifier(torch.nn.Module):
    def __init__(self, layers):
        super(NN_BinClassifier, self).__init__()

        net_topology = OrderedDict()
        for i in range(len(layers)):
            if i != len(layers)-1:
                net_topology[f'fc{i}'] = torch.nn.Linear(layers[i], layers[i+1])
                net_topology[f'LeakyReLu{i}'] = torch.nn.LeakyReLU()
            else:
                # last layer (output)
                net_topology[f'fc{i}'] = torch.nn.Linear(layers[i], 1)
                net_topology[f'Sigmoid'] = torch.nn.Sigmoid()
        
        self.net = torch.nn.Sequential(net_topology)

    def forward(self, x):
        out = self.net(x)
        return out

In [4]:
import torch.optim as optim

net = NN_BinClassifier([17, 7, 2])
criterion = torch.nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(X_train.shape[0]):
        # get the inputs; data is a list of [inputs, labels]
        inputs = torch.from_numpy(X_train[i,:])
        labels = torch.from_numpy(np.atleast_1d(y_train[i]))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: expected scalar type Long but found Float